In [136]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import smtplib
import csv
import numpy as np

In [138]:
URL = 'https://www.amazon.com/anime-figures/s?k=anime+figures&s=review-rank&qid=1683064266&ref=sr_st_review-rank&ds=v1%3ABx1cg87n%2BlVShIP8zxXOzss3FhCLd1ThObswAFkmg%2F0'

In [140]:
# Headers for request
HEADERS = ({'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'}) #add your user agent 

In [142]:
# HTTP Request
webpage = requests.get(URL, headers=HEADERS)

In [145]:
# Soup Object containiang all data
soup = BeautifulSoup(webpage.content, "html.parser")

In [147]:
# Fetch links as List of Tag Objects
links = soup.find_all("a", attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

In [149]:
links

[<a class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal" href="/Taito-Re-Starting-Another-Precious/dp/B086HQ34LJ/ref=sr_1_1?keywords=anime+figures&amp;qid=1683148873&amp;sr=8-1"><span class="a-size-base-plus a-color-base a-text-normal">Taito Re: Zero -Starting Life in Another World-: Rem Precious Figure (Fluffy Parka Version)</span> </a>,
 <a class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal" href="/Square-Enix-Replicant-1-22474487139-Nendoroid/dp/B09G6KPRP1/ref=sr_1_2?keywords=anime+figures&amp;qid=1683148873&amp;sr=8-2"><span class="a-size-base-plus a-color-base a-text-normal">Square Enix NieR Replicant Ver. 1.22474487139. Kainé Nendoroid Action Figure</span> </a>,
 <a class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal" href="/Funko-Star-Wars-Mandalorian-Multicolour/dp/B07W8JRFXD/ref=sr_1_3?keywords=anime+figures&amp;qid=1683148873&amp;sr=8-3"><span class="a-size-base-plus a-co

In [166]:
link = links[0].get('href')

In [167]:
product_list = "https://amazon.com" + link

In [168]:
product_list

'https://amazon.com/Taito-Re-Starting-Another-Precious/dp/B086HQ34LJ/ref=sr_1_1?keywords=anime+figures&qid=1683148884&sr=8-1'

In [169]:
new_webpage = requests.get(product_list, headers=HEADERS)

In [170]:
new_webpage

<Response [200]>

In [171]:
# Soup Object containiang all data
new_soup = BeautifulSoup(new_webpage.content, "html.parser")

In [172]:
title = new_soup.find("span", attrs={"id":'productTitle'}).text.strip()

In [173]:
price = new_soup.find("span", attrs={"class":'a-offscreen'}).text

In [179]:
rating = new_soup.find("span", attrs={'class':'a-icon-alt'}).text

In [180]:
reviews= new_soup.find("span", attrs={'id':'acrCustomerReviewText'}).text

In [181]:
print(title, price, rating, review_count)

Taito Re: Zero -Starting Life in Another World-: Rem Precious Figure (Fluffy Parka Version) $31.66 4.9 out of 5 stars 87 ratings


In [182]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={"class":'a-offscreen'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

In [183]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = 'https://www.amazon.com/anime-figures/s?k=anime+figures&s=review-rank&qid=1683064266&ref=sr_st_review-rank&ds=v1%3ABx1cg87n%2BlVShIP8zxXOzss3FhCLd1ThObswAFkmg%2F0'

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [185]:
amazon_df

,title,price,rating,reviews,availability
0,Taito Re: Zero -Starting Life in Another World...,$31.66,5.0 out of 5 stars,87 ratings,Only 1 left in stock - order soon
1,Square Enix NieR Replicant Ver. 1.22474487139....,$50.06,5.0 out of 5 stars,86 ratings,Only 3 left in stock - order soon
2,Funko Pop! Star Wars: The Mandalorian Action F...,$8.78,4.5 out of 5 stars,"31,273 ratings",In Stock
3,Funko Pop! Games: Pokémon - Mewtwo Vinyl Figur...,$14.00,4.9 out of 5 stars,"7,844 ratings",In Stock
4,Funko Pop! Games: Pokemon - Psyduck Collectibl...,$12.00,4.9 out of 5 stars,"7,642 ratings",In Stock
5,"Funko Pop! Deluxe, Marvel: Avengers Victory Sh...",$5.85,4.9 out of 5 stars,"6,695 ratings",In Stock
6,Funko 42937 POP. Vinyl: Animation: My Hero Aca...,$6.40,4.9 out of 5 stars,"7,007 ratings",In Stock
7,Funko POP Rocks: Slash Action Figure,$18.85,4.9 out of 5 stars,"6,791 ratings",In Stock
8,Banpresto Dragon Ball Super World Figure Colos...,$84.99,5.0 out of 5 stars,75 ratings,Only 16 left in stock - order soon
9,Funko Pop! Games: Pokemon - Vaporeon Vinyl Figure,$11.30,4.9 out of 5 stars,"5,018 ratings",In Stock
